# Lesson 5: Integrating MCP with LLMs for Game NPCs

In this lesson, you'll learn how to connect LLM-powered NPCs with external knowledge using MCP (Model Context Protocol) and DeepWiki.

## 🌐 What is MCP?

MCP allows LLMs to access external tools and knowledge sources, such as DeepWiki, to answer questions or enhance in-game dialogue.

## 🛠️ MCP + LLM Setup

We'll use OpenAI's Python SDK and connect to DeepWiki via MCP.
- https://deepwiki.com/
- https://mcpmarket.com/

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

# Example: Ask DeepWiki via MCP
resp = client.responses.create(
    model="gpt-4.1",
    tools=[
        {
            "type": "mcp",
            "server_label": "deepwiki",
            "server_url": "https://mcp.deepwiki.com/mcp",
            "require_approval": {
                "never": {
                    "tool_names": ["ask_question", "read_wiki_structure"]
                }
            }
        },
    ],
    input="What transport protocols does the 2025-03-26 version of the MCP spec (modelcontextprotocol/modelcontextprotocol) support?",
)

print(resp.output_text)

The 2025-03-26 version of the Model Context Protocol (MCP) specification (modelcontextprotocol/modelcontextprotocol) supports the following transport protocols:

1. **stdio**:
   - The MCP server is launched as a subprocess.
   - Communication uses standard input (stdin) and standard output (stdout), with each message delimited by a newline.
   - Standard error (stderr) can be used for server-side logs.
   
2. **Streamable HTTP**:
   - The MCP server runs as a separate process.
   - Clients connect via HTTP POST and GET requests.
   - The protocol optionally supports Server-Sent Events (SSE) for streaming multiple messages from the server.
   - Streamable HTTP replaces the older "HTTP+SSE" transport from previous spec versions.

3. **Custom Transports**:
   - The MCP protocol is transport agnostic. Clients and servers can implement custom transports as long as they:
     - Preserve the JSON-RPC 2.0 message format,
     - Adhere to MCP’s message lifecycle requirements.

**All MCP transp

## 🎮 Simple Game State and NPC

We'll create a minimal game state and a single NPC that can answer questions using LLM + MCP.

In [4]:
from dataclasses import dataclass, field
from typing import Dict, Any

@dataclass
class SimpleNPC:
    name: str
    personality: str
    knowledge_domains: list

    def ask(self, question: str) -> str:
        resp = client.responses.create(
            model="gpt-4o-mini",
            tools=[
                {
                    "type": "mcp",
                    "server_label": "deepwiki",
                    "server_url": "https://mcp.deepwiki.com/mcp",
                    "require_approval": {
                        "never": {
                            "tool_names": ["ask_question", "read_wiki_structure"]
                        }
                    }
                },
            ],
            input=question,
        )
        return resp.output_text

In [5]:
# Minimal game loop: player can ask the NPC anything, and the NPC will use MCP+LLM to answer.
npc = SimpleNPC(
    name="บรรณารักษ์เงา",
    personality="รักความรู้ ช่างสังเกต",
    knowledge_domains=["ประวัติศาสตร์", "ความลับของกาลเวลา"]
)

print(f"คุณพบ {npc.name} (NPC): {npc.personality}")
while True:
    q = input("ถามคำถามกับ NPC (หรือพิมพ์ 'exit'): ")
    print(f"\nคุณถาม: {q}\n")
    if q.strip().lower() == 'exit':
        print("ลาก่อน!")
        break
    print(f"{npc.name} กำลังค้นคว้าคำตอบ...\n")
    answer = npc.ask(q)
    print(f"{npc.name}: {answer}\n")

คุณพบ บรรณารักษ์เงา (NPC): รักความรู้ ช่างสังเกต
บรรณารักษ์เงา กำลังค้นคว้าคำตอบ...

บรรณารักษ์เงา: นาคนคร เป็นชื่อของเมืองในตำนานหรือเขตที่เกี่ยวข้องกับความเชื่อและปรัชญาในวรรณกรรมไทย ซึ่งมักมีการกล่าวถึงในเรื่องที่เกี่ยวข้องกับความเป็นอมตะและพลังของกาลเวลา เมืองนี้มักถูกมองว่าเป็นที่สำหรับผู้ที่ต้องการหลีกหนีจากความทุกข์ในโลกมนุษย์ และเป็นแหล่งรวมของความรู้และพลังที่เหนือธรรมชาติ

ในแง่ของกาลเวลา นาคนครอาจสื่อถึงความอมตะหรือการเบี่ยงเบนของเวลา เช่น ผู้ที่อาศัยอยู่ในนาคนครอาจไม่ต้องเผชิญกับการแก่ชรา หรือสามารถควบคุมเวลาได้ในบางระดับ นอกจากนั้นยังเป็นสัญลักษณ์ของการเดินทางในจิตใจและการค้นหาความหมายในชีวิต ซึ่งเป็นแนวทางที่มีความเกี่ยวข้องกับการเข้าใจธรรมชาติของเวลาในเชิงปรัชญาอีกด้วย

หากคุณต้องการข้อมูลเพิ่มเติมในแง่มุมใดเป็นพิเศษ แจ้งได้นะ!

ลาก่อน!


## 🏆 Challenge

- ลองถาม NPC เกี่ยวกับหัวข้อที่ลึกซึ้งหรือเฉพาะทาง
- สังเกตว่า NPC สามารถใช้ DeepWiki เพื่อค้นหาคำตอบได้
- ขยาย NPC หรือเกมให้รองรับหลาย NPC ที่ใช้ MCP ต่างกัน